In [2]:
import numpy as np
import pandas as pd
from sklearn.svm import SVC
from sklearn.model_selection import KFold
import matplotlib.pyplot as plt
from sklearn import datasets
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer
%matplotlib inline

Одна из причин популярности линейных методов заключается в том, что они хорошо работают на разреженных данных. Так называются выборки с большим количеством признаков, где на каждом объекте большинство признаков равны нулю. Разреженные данные возникают, например, при работе с текстами. Дело в том, что текст удобно кодировать с помощью "мешка слов" — формируется столько признаков, сколько всего уникальных слов встречается в текстах, и значение каждого признака равно числу вхождений в документ соответствующего слова. Ясно, что общее число различных слов в наборе текстов может достигать десятков тысяч, и при этом лишь небольшая их часть будет встречаться в одном конкретном тексте.

Можно кодировать тексты хитрее, и записывать не количество вхождений слова в текст, а TF-IDF. Это показатель, который равен произведению двух чисел: TF (term frequency) и IDF (inverse document frequency). Первая равна отношению числа вхождений слова в документ к общей длине документа. Вторая величина зависит от того, в скольки документах выборки встречается это слово. Чем больше таких документов, тем меньше IDF. Таким образом, TF-IDF будет иметь высокое значение для тех слов, которые много раз встречаются в данном документе, и редко встречаются в остальных.
Данные

Как мы уже говорили выше, линейные методы часто применяются для решения различных задач анализа текстов. В этом задании мы применим метод опорных векторов для определения того, к какой из тематик относится новость: атеизм или космос.
Реализация в Scikit-Learn

Для начала вам потребуется загрузить данные. В этом задании мы воспользуемся одним из датасетов, доступных в scikit-learn'е — 20 newsgroups. Для этого нужно воспользоваться модулем datasets:

from sklearn import datasets

newsgroups = datasets.fetch_20newsgroups(
                    subset='all', 
                    categories=['alt.atheism', 'sci.space']
             )
             
После выполнения этого кода массив с текстами будет находиться в поле newsgroups.data, номер класса — в поле newsgroups.target.

Одна из сложностей работы с текстовыми данными состоит в том, что для них нужно построить числовое представление. Одним из способов нахождения такого представления является вычисление TF-IDF. В Scikit-Learn это реализовано в классе sklearn.feature_extraction.text.TfidfVectorizer. Преобразование обучающей выборки нужно делать с помощью функции fit_transform, тестовой — с помощью transform.

Реализация SVM-классификатора находится в классе sklearn.svm.SVC. Веса каждого признака у обученного классификатора хранятся в поле coef_. Чтобы понять, какому слову соответствует i-й признак, можно воспользоваться методом get_feature_names() у TfidfVectorizer:

feature_mapping = vectorizer.get_feature_names()
print feature_mapping[i]

Подбор параметров удобно делать с помощью класса sklearn.grid_search.GridSearchCV (При использовании библиотеки scikit-learn версии 18.0.1 sklearn.model_selection.GridSearchCV). Пример использования:

grid = {'C': np.power(10.0, np.arange(-5, 6))}
cv = KFold(y.size, n_folds=5, shuffle=True, random_state=241)
clf = svm.SVC(kernel='linear', random_state=241)
gs = GridSearchCV(clf, grid, scoring='accuracy', cv=cv)
gs.fit(X, y)

При использовании библиотеки scikit-learn версии 18.0.1 и выше KFold задаётся немного по-другому:
cv = KFold(n_splits=5, shuffle=True, random_state=241)

Первым аргументом в GridSearchCV передается классификатор, для которого будут подбираться значения параметров, вторым — словарь (dict), задающий сетку параметров для перебора. После того, как перебор окончен, можно проанализировать значения качества для всех значений параметров и выбрать наилучший вариант:

for a in gs.grid_scores_:
    # a.mean_validation_score — оценка качества по кросс-валидации
    # a.parameters — значения параметров
    
Инструкция по выполнению

    Загрузите объекты из новостного датасета 20 newsgroups, относящиеся к категориям "космос" и "атеизм" (инструкция приведена выше). Обратите внимание, что загрузка данных может занять несколько минут

    Вычислите TF-IDF-признаки для всех текстов. Обратите внимание, что в этом задании мы предлагаем вам вычислить TF-IDF по всем данным. При таком подходе получается, что признаки на обучающем множестве используют информацию из тестовой выборки — но такая ситуация вполне законна, поскольку мы не используем значения целевой переменной из теста. На практике нередко встречаются ситуации, когда признаки объектов тестовой выборки известны на момент обучения, и поэтому можно ими пользоваться при обучении алгоритма.

    Подберите минимальный лучший параметр C из множества [10^-5, 10^-4, ... 10^4, 10^5] для SVM с линейным ядром (kernel='linear') при помощи кросс-валидации по 5 блокам. Укажите параметр random_state=241 и для SVM, и для KFold. В качестве меры качества используйте долю верных ответов (accuracy).

    Обучите SVM по всей выборке с оптимальным параметром C, найденным на предыдущем шаге.

    Найдите 10 слов с наибольшим абсолютным значением веса (веса хранятся в поле coef_ у svm.SVC). Они являются ответом на это задание. Укажите эти слова через запятую или пробел, в нижнем регистре, в лексикографическом порядке.

In [2]:
newsgroups = datasets.fetch_20newsgroups(subset='all', categories=['alt.atheism', 'sci.space'])

In [3]:
#newsgroups.data
y = newsgroups.target

In [7]:
TfidfVectorizer?
#Преобразование обучающей выборки нужно делать с помощью функции fit_transform, тестовой — с помощью transform.

In [4]:
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(newsgroups.data)
#print(vectorizer.get_feature_names())

In [11]:
vectorizer?

In [5]:
grid = {'C': np.power(10.0, np.arange(-5, 6))}
cv = KFold(n_splits=5, shuffle=True, random_state=241)
clf = SVC(kernel='linear', random_state=241)
gs = GridSearchCV(clf, grid, scoring='accuracy', cv=cv)
gs.fit(X, y)

GridSearchCV(cv=KFold(n_splits=5, random_state=241, shuffle=True),
             estimator=SVC(kernel='linear', random_state=241),
             param_grid={'C': array([1.e-05, 1.e-04, 1.e-03, 1.e-02, 1.e-01, 1.e+00, 1.e+01, 1.e+02,
       1.e+03, 1.e+04, 1.e+05])},
             scoring='accuracy')

In [7]:
grid

{'C': array([1.e-05, 1.e-04, 1.e-03, 1.e-02, 1.e-01, 1.e+00, 1.e+01, 1.e+02,
        1.e+03, 1.e+04, 1.e+05])}

In [32]:
GridSearchCV?

In [26]:
for a in gs.grid_scores_:
    # a.mean_validation_score — оценка качества по кросс-валидации
    # a.parameters — значения параметров
    print(a.parameters)

AttributeError: 'GridSearchCV' object has no attribute 'grid_scores_'

In [8]:
gs.best_params_

{'C': 1.0}

In [36]:
gs.best_params_['C']

1.0

In [33]:
pd.DataFrame(gs.cv_results_)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_C,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,4.359813,0.201477,1.049717,0.040726,0.00001,{'C': 1e-05},0.544693,0.579832,0.571429,0.501401,0.565826,0.552636,0.028124,8
1,4.223458,0.099030,1.016188,0.021366,0.0001,{'C': 0.0001},0.544693,0.579832,0.571429,0.501401,0.565826,0.552636,0.028124,8
2,4.358517,0.084997,1.010107,0.018529,0.001,{'C': 0.001},0.544693,0.579832,0.571429,0.501401,0.565826,0.552636,0.028124,8
3,4.475308,0.145986,1.087158,0.067749,0.01,{'C': 0.01},0.544693,0.579832,0.571429,0.501401,0.565826,0.552636,0.028124,8
4,3.604141,0.096159,0.866028,0.021691,0.1,{'C': 0.1},0.958101,0.949580,0.957983,0.935574,0.949580,0.950164,0.008218,7
5,2.169620,0.024029,0.506953,0.023114,1.0,{'C': 1.0},0.994413,0.985994,1.000000,0.991597,0.994398,0.993280,0.004552,1
6,2.136016,0.033425,0.504461,0.017035,10.0,{'C': 10.0},0.994413,0.985994,1.000000,0.991597,0.994398,0.993280,0.004552,1
7,2.120512,0.032749,0.498414,0.010957,100.0,{'C': 100.0},0.994413,0.985994,1.000000,0.991597,0.994398,0.993280,0.004552,1
8,2.201697,0.060111,0.509836,0.006536,1000.0,{'C': 1000.0},0.994413,0.985994,1.000000,0.991597,0.994398,0.993280,0.004552,1
9,2.158285,0.032323,0.507053,0.016976,10000.0,{'C': 10000.0},0.994413,0.985994,1.000000,0.991597,0.994398,0.993280,0.004552,1


In [3]:
clf = SVC(C=1.0, kernel='linear', random_state=241)
clf.fit(X, y)

NameError: name 'X' is not defined

In [38]:
#Найдите 10 слов с наибольшим абсолютным значением веса (веса хранятся в поле coef_ у svm.SVC). 
#Они являются ответом на это задание. Укажите эти слова через запятую или пробел, 
#в нижнем регистре, в лексикографическом порядке.
cf = list(clf.coef_)
print(type(cf))
print(cf)

<class 'list'>
[<1x28382 sparse matrix of type '<class 'numpy.float64'>'
	with 18404 stored elements in Compressed Sparse Row format>]


In [4]:
#SVC?
#cf = list(cf)

clf?

# coef_ : ndarray of shape (n_classes * (n_classes - 1) / 2, n_features)
#     Weights assigned to the features (coefficients in the primal
#     problem). This is only available in the case of a linear kernel.

#     `coef_` is a readonly property derived from `dual_coef_` and
#     `support_vectors_`.

In [31]:
X_ex = np.array([[-1, -1], [-2, -1], [1, 1], [2, 1]])
y_ex = np.array([1, 1, 2, 2]) # 2 classes

clf_ex = SVC(kernel='linear')
clf_ex.fit(X_ex, y_ex)

print(clf_ex.coef_)
clf_ex.coef_.shape

[[0.5 0.5]]


(1, 2)

In [47]:
word = pd.DataFrame(data=vectorizer.get_feature_names())
coef = pd.DataFrame(data=np.abs(np.asarray(clf.coef_.todense()).reshape(-1)))
data = pd.concat([word, coef], axis=1)
data.columns = ['word', 'coef']
#data.sort_index(by=['coef'])[-10:]

In [48]:
data

,word,coef
0,00,0.292581
1,000,0.123148
2,0000,0.000000
3,00000,0.000000
4,000000,0.000000
...,...,...
28377,zwakke,0.009864
28378,zware,0.009864
28379,zwarte,0.019729
28380,zwork,0.058313


In [55]:
ix = np.argsort(np.abs(np.asarray(clf.coef_.todense())).reshape(-1))[-10:]

In [68]:
for i in ix: print(vectorizer.get_feature_names()[i])

sci
keith
bible
religion
sky
moon
atheists
atheism
god
space


In [67]:
ans = []
for i in ix: ans.append(vectorizer.get_feature_names()[i])
ans.sort()
answer = ''
for a in ans: answer += a + ' '
answer = answer.strip()
print(answer)
with open('lab7.txt', 'w') as outfile:
    outfile.write(answer)

atheism atheists bible god keith moon religion sci sky space
